In [1]:
import numpy as np
import os
from google.colab import drive
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'google.colab'

In [7]:
drive.mount('/content/drive')
dataset_path = "/content/drive/My Drive/Colab Notebooks/Agrar_uchun/v"

class_folders = [os.path.join(dataset_path, folder) for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

X = []  # Xususiyatlar
y = []  # Etiketkalar

model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
model_res_net = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


for label, folder in enumerate(class_folders):  # Har bir sinf uchun
    for image_file in os.listdir(folder):
        img = Image.open(os.path.join(folder, image_file))# Tasvir 256x256 o'lchamga ega
        img_array = np.array(img)  # Tasvirni numpy massiviga aylantirish
        img_preprocessed = preprocess_input(img_array[np.newaxis, ...])  # Tasvirni normalizatsiya qilish
        features = model_res_net.predict(img_preprocessed).flatten()
        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Shape of X: {X.shape}")
print(f"Labels (y): {y}")

Mounted at /content/drive
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━

In [8]:
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [9]:
print(f"Shape of X: {X_normalized.shape}")
print(f"Labels (y): {y.shape}")

Shape of X: (1000, 131072)
Labels (y): (1000,)


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Ma'lumotlarni bo'lish
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

param_grid = {'n_neighbors': range(1, 21)}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
best_accuracy = grid_search.best_score_

print(f"Best k: {best_k}")
print(f"Best cross-validated accuracy: {best_accuracy * 100:.2f}%")

# KNN modelini yaratish
KNN = KNeighborsClassifier(n_neighbors=best_k)
KNN.fit(X_train, y_train)

# Modelni sinash
accuracy = KNN.score(X_test, y_test)
print(f"KNN model accuracy: {accuracy * 100:.2f}%")




Best k: 1
Best cross-validated accuracy: 30.12%
KNN model accuracy: 36.00%


In [11]:
print(f"Shape of X: {X_train.shape}")
print(f"Labels (y): {y}")


Shape of X: (800, 131072)
Labels (y): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)  # Faqat 100 asosiy xususiyat
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train_pca, y_train)

accuracy = knn.score(X_test_pca, y_test)
print(f"Accuracy after PCA: {accuracy * 100:.2f}%")
print(f"Shape of X: {X_train_pca.shape}")
print(f"Labels (y): {y}")


Accuracy after PCA: 65.50%
Shape of X: (800, 100)
Labels (y): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

In [ ]:
from collections import Counter

print(Counter(y_train))


Counter({1: 87, 7: 87, 4: 85, 0: 80, 9: 79, 3: 79, 5: 78, 8: 77, 6: 75, 2: 73})


K-NN algoritmida VGG16 modelining qo'llanilishi.

In [ ]:
import os
import numpy as np
from google.colab import drive
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.decomposition import PCA
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import accuracy_score

knn_vgg16 = KNeighborsClassifier(n_neighbors=1)
knn_pca = KNeighborsClassifier(n_neighbors=1)
knn_res = KNeighborsClassifier(n_neighbors=1)

model_res_net = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# 1. Data Preparation
def load_images_from_directory(directory, target_size=(256, 256)):
    images = []
    labels = []
    for label in os.listdir(directory):
        class_dir = os.path.join(directory, label)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                img = load_img(img_path, target_size=target_size)
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

# 2. Data Generators (Augmentation)
def create_data_generators(train_images, train_labels, val_images, val_labels, batch_size=32):
    train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = train_gen.flow(train_images, train_labels, batch_size=batch_size)
    val_generator = val_gen.flow(val_images, val_labels, batch_size=batch_size)

    return train_generator, val_generator

# 3. Using VGG16 to extract features
def load_vgg16_model():
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    return vgg16

# 4. Feature Extraction Function
def extract_features(model, images):
    features = model.predict(images)
    return features.reshape(features.shape[0], -1)

# Main function
def main(data_dir):
    # Load images and labels
    images, labels = load_images_from_directory(data_dir)

    # Normalize images
    images = images / 255.0

    # Encode labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

    # # Load VGG16 model
    # vgg16_model = load_vgg16_model()

    # # Extract features from training and validation data
    # train_X = extract_features(vgg16_model, preprocess_input(X_train))
    # test_X = extract_features(vgg16_model, preprocess_input(X_test))

    # Extract features from training and validation data
    X_train_res = extract_features(model_res_net, preprocess_input(X_train))
    X_test_res = extract_features(model_res_net, preprocess_input(X_test))

    # 5. K-NN Classification

    # knn_vgg16.fit(train_X, y_train)################################################################################################

    # # Validate the model
    # accuracy = knn_vgg16.score(test_X, y_test)
    # print(f"Validation Accuracy: {accuracy * 100:.2f}%")

    # pca = PCA(n_components=100)  # Faqat 100 asosiy xususiyat
    # X_train_pca = pca.fit_transform(train_X)
    # X_test_pca = pca.transform(test_X)


    # knn_pca.fit(X_train_pca, y_train)####################################################################################################

    # accuracy_pca = knn_pca.score(X_test_pca, y_test)
    # print(f"Accuracy after PCA: {accuracy_pca * 100:.2f}%")


    res_pca = PCA(n_components=100)  # Faqat 100 asosiy xususiyat
    X_train_res_pca = res_pca.fit_transform(X_train_res)
    X_test_res_pca = res_pca.transform(X_test_res)

    knn_res.fit(X_train_res_pca, y_train)####################################################################################################

    # Validate the model
    accuracy_res_pca = knn_res.score(X_test_res_pca, y_test)
    print(f"Validation Accuracy: {accuracy_res_pca * 100:.2f}%")

    # print(f"Shape of train_X: {train_X.shape}")
    # print(f"Labels (y_train): {y_train}")

    # print(f"Shape of X_train_pca: {X_train_pca.shape}")
    # print(f"Labels (y_test): {y_test}")

    print(f"Shape of X_train_res_pca: {X_train_res_pca.shape}")
    print(f"Labels (y_test): {y_train}")

# Directory of the dataset
# Google Drive ni ulash
drive.mount('/content/drive')
data_dir = "/content/drive/My Drive/Colab Notebooks/Agrar_uchun/v"
main(data_dir)



[[-2.82842712]
 [ 0.        ]
 [ 2.82842712]]
